# Notebook 04: Multi-Hop Path Discovery

## Objective

Chain one-hop calls for 2+ hop paths between entities.

## BFS Explosion Safeguards

Graph traversal can explode combinatorially. This notebook implements:

| Safeguard | Value | Purpose |
|-----------|-------|---------|
| `MAX_VISITED_NODES` | 1000 | Hard cap on explored nodes |
| `TIMEOUT_SECONDS` | 30 | Per-query time limit |
| `MAX_QUEUE_SIZE` | 5000 | Prevent memory exhaustion |
| `SEMANTIC_PREDICATE_PRIORITY` | True | Expand semantic edges before equivalency |

In [1]:
# Standard imports
import sys
import time
from pathlib import Path
from datetime import datetime
from collections import Counter

# Add project root to path
PROJECT_ROOT = Path.cwd().parents[1]
sys.path.insert(0, str(PROJECT_ROOT / 'src'))
sys.path.insert(0, str(Path.cwd()))

# Import utilities
from kg_o1_v3_utils import (
    test_one_hop, find_path_bfs,
    hybrid_search, text_search,
    save_json, load_json,
    MultiHopPath, SemanticTriple, PathFindingResult,
    MAX_VISITED_NODES, TIMEOUT_SECONDS, MAX_QUEUE_SIZE,
)

# Output directory
OUTPUT_DIR = Path.cwd() / 'outputs'
OUTPUT_DIR.mkdir(exist_ok=True)

print(f"Project root: {PROJECT_ROOT}")
print(f"Output directory: {OUTPUT_DIR}")
print(f"\nSafeguards:")
print(f"  MAX_VISITED_NODES: {MAX_VISITED_NODES}")
print(f"  TIMEOUT_SECONDS: {TIMEOUT_SECONDS}")
print(f"  MAX_QUEUE_SIZE: {MAX_QUEUE_SIZE}")

Project root: /home/trentleslie/Insync/projects/biomapper2
Output directory: /home/trentleslie/Insync/projects/biomapper2/notebooks/kg_o1_v3/outputs

Safeguards:
  MAX_VISITED_NODES: 1000
  TIMEOUT_SECONDS: 30
  MAX_QUEUE_SIZE: 5000


## 1. Load Subgraph Data

Use entities from NB03 that have semantic relations.

In [2]:
# Load subgraphs from NB03
subgraph_file = OUTPUT_DIR / 'semantic_subgraphs.json'

if subgraph_file.exists():
    subgraph_data = load_json(subgraph_file)
    subgraphs = subgraph_data.get('subgraphs', [])
    
    # Get entities that have relations
    entities_with_relations = [
        (sg['center_entity_id'], sg['center_entity_name'])
        for sg in subgraphs
        if sg.get('outgoing_relations') or sg.get('incoming_relations')
    ]
    
    print(f"Loaded {len(subgraphs)} subgraphs from NB03")
    print(f"Entities with semantic relations: {len(entities_with_relations)}")
else:
    print("WARNING: NB03 output not found. Using default entities.")
    entities_with_relations = [
        ("CHEBI:4167", "glucose"),
        ("CHEBI:15846", "NAD+"),
        ("CHEBI:30616", "ATP"),
    ]

Loaded 20 subgraphs from NB03
Entities with semantic relations: 20


## 2. Define Path-Finding Test Cases

Create pairs of entities to find paths between.

In [3]:
# Generate entity pairs for path finding
import itertools

# Take first 10 entities and create pairs
test_entities = entities_with_relations[:10]

# Create all pairs (avoid self-loops)
entity_pairs = [
    (e1, e2) for e1, e2 in itertools.combinations(test_entities, 2)
]

print(f"Test entity pairs: {len(entity_pairs)}")
print("\nSample pairs:")
for (id1, name1), (id2, name2) in entity_pairs[:5]:
    print(f"  {name1} <--> {name2}")

Test entity pairs: 45

Sample pairs:
  glucose <--> NAD+
  glucose <--> cholesterol
  glucose <--> alanine
  glucose <--> ATP
  glucose <--> water


## 3. Find Multi-Hop Paths

Use BFS with safeguards to find paths between entity pairs.

In [4]:
# Find paths between entity pairs
print("="*60)
print("Finding multi-hop paths")
print("="*60)

path_results = []
termination_reasons = Counter()

# Limit to first 20 pairs to avoid very long runtime
max_pairs = min(20, len(entity_pairs))

for i, ((start_id, start_name), (end_id, end_name)) in enumerate(entity_pairs[:max_pairs]):
    print(f"\n[{i+1}/{max_pairs}] {start_name} --> {end_name}")
    
    # Try with semantic_only=True first
    result = find_path_bfs(
        start_id, end_id,
        max_hops=3,
        semantic_only=True,
    )
    
    termination_reasons[result.termination_reason] += 1
    
    path_data = {
        'start_id': start_id,
        'start_name': start_name,
        'end_id': end_id,
        'end_name': end_name,
        'path_length': len(result.path),
        'path': [{
            'subject': t.subject_name,
            'predicate': t.predicate,
            'object': t.object_name,
        } for t in result.path],
        'stats': result.stats,
        'termination_reason': result.termination_reason,
    }
    path_results.append(path_data)
    
    if result.path:
        print(f"  FOUND! Path length: {len(result.path)}")
        for t in result.path:
            print(f"    {t.subject_name} --[{t.predicate}]--> {t.object_name}")
    else:
        print(f"  Not found: {result.termination_reason}")
        print(f"  Stats: visited={result.stats.get('nodes_visited', 0)}, "
              f"api_calls={result.stats.get('api_calls', 0)}")

Finding multi-hop paths

[1/20] glucose --> NAD+
  Not found: exhausted
  Stats: visited=419, api_calls=76

[2/20] glucose --> cholesterol


  Not found: exhausted
  Stats: visited=419, api_calls=76

[3/20] glucose --> alanine
  Not found: exhausted
  Stats: visited=419, api_calls=76

[4/20] glucose --> ATP


  Not found: exhausted
  Stats: visited=419, api_calls=76

[5/20] glucose --> water
  FOUND! Path length: 3
    CHEBI:4167 --[biolink:subclass_of]--> D-glucose
    CHEBI:17634 --[biolink:has_part]--> royal jelly
    CHEBI:78665 --[biolink:has_part]--> water

[6/20] glucose --> L-leucine
  Not found: exhausted
  Stats: visited=419, api_calls=76

[7/20] glucose --> L-tryptophan


  Not found: exhausted
  Stats: visited=419, api_calls=76

[8/20] glucose --> L-tyrosine
  Not found: exhausted
  Stats: visited=419, api_calls=76

[9/20] glucose --> L-valine


  Not found: exhausted
  Stats: visited=419, api_calls=76

[10/20] NAD+ --> cholesterol
  Not found: exhausted
  Stats: visited=637, api_calls=82

[11/20] NAD+ --> alanine


  Not found: exhausted
  Stats: visited=637, api_calls=82

[12/20] NAD+ --> ATP
  Not found: exhausted
  Stats: visited=637, api_calls=82

[13/20] NAD+ --> water
  FOUND! Path length: 3
    CHEBI:15846 --[biolink:related_to]--> NADH
    CHEBI:16908 --[biolink:related_to]--> ALDH1A1
    NCBIGene:216 --[biolink:interacts_with]--> water

[14/20] NAD+ --> L-leucine
  FOUND! Path length: 3
    CHEBI:15846 --[biolink:related_to]--> NADH
    CHEBI:16908 --[biolink:related_to]--> GLUD1
    NCBIGene:2746 --[biolink:interacts_with]--> L-glutamic acid

[15/20] NAD+ --> L-tryptophan


  Not found: exhausted
  Stats: visited=637, api_calls=82

[16/20] NAD+ --> L-tyrosine
  Not found: exhausted
  Stats: visited=637, api_calls=82

[17/20] NAD+ --> L-valine


  Not found: exhausted
  Stats: visited=637, api_calls=82

[18/20] cholesterol --> alanine
  Not found: exhausted
  Stats: visited=797, api_calls=117

[19/20] cholesterol --> ATP


  Not found: exhausted
  Stats: visited=797, api_calls=117

[20/20] cholesterol --> water
  Not found: exhausted
  Stats: visited=797, api_calls=117


In [5]:
# Summary of path finding
print("\n" + "="*60)
print("PATH FINDING SUMMARY")
print("="*60)

found_paths = [r for r in path_results if r['path_length'] > 0]

print(f"\nTotal pairs tested: {len(path_results)}")
print(f"Paths found: {len(found_paths)} ({100*len(found_paths)/len(path_results):.1f}%)")

print(f"\nTermination reasons:")
for reason, count in termination_reasons.most_common():
    pct = 100 * count / len(path_results)
    print(f"  {reason}: {count} ({pct:.1f}%)")

if found_paths:
    avg_length = sum(r['path_length'] for r in found_paths) / len(found_paths)
    print(f"\nAverage path length: {avg_length:.1f}")
    
    print(f"\nPath length distribution:")
    length_counts = Counter(r['path_length'] for r in found_paths)
    for length, count in sorted(length_counts.items()):
        print(f"  {length} hops: {count} paths")


PATH FINDING SUMMARY

Total pairs tested: 20
Paths found: 3 (15.0%)

Termination reasons:
  exhausted: 17 (85.0%)
  found: 3 (15.0%)

Average path length: 3.0

Path length distribution:
  3 hops: 3 paths


## 4. Analyze Explosion Behavior

In [6]:
# Analyze explosion behavior
print("="*60)
print("EXPLOSION ANALYSIS")
print("="*60)

all_stats = [r['stats'] for r in path_results]

# Calculate aggregate statistics
total_nodes_visited = sum(s.get('nodes_visited', 0) for s in all_stats)
total_api_calls = sum(s.get('api_calls', 0) for s in all_stats)
total_edges_examined = sum(s.get('edges_examined', 0) for s in all_stats)
total_semantic_edges = sum(s.get('semantic_edges', 0) for s in all_stats)
total_equiv_edges = sum(s.get('equivalency_edges', 0) for s in all_stats)

print(f"\nAggregate statistics:")
print(f"  Total nodes visited: {total_nodes_visited}")
print(f"  Total API calls: {total_api_calls}")
print(f"  Total edges examined: {total_edges_examined}")
print(f"  Semantic edges: {total_semantic_edges}")
print(f"  Equivalency edges: {total_equiv_edges}")

# Per-query averages
n_queries = len(path_results)
print(f"\nPer-query averages:")
print(f"  Avg nodes visited: {total_nodes_visited/n_queries:.1f}")
print(f"  Avg API calls: {total_api_calls/n_queries:.1f}")
print(f"  Avg edges examined: {total_edges_examined/n_queries:.1f}")

# Timeout rate
timeout_rate = termination_reasons.get('timeout', 0) / n_queries
print(f"\nTimeout rate: {100*timeout_rate:.1f}%")

# Recommendation
if timeout_rate > 0.1:
    print(f"\nWARNING: High timeout rate. Consider reducing max_hops or using semantic_only.")
elif timeout_rate > 0.05:
    print(f"\nNote: Moderate timeout rate. Current safeguards are working.")
else:
    print(f"\nGood: Low timeout rate. Safeguards are effective.")

EXPLOSION ANALYSIS

Aggregate statistics:
  Total nodes visited: 9662
  Total API calls: 1548
  Total edges examined: 92515
  Semantic edges: 78618
  Equivalency edges: 187

Per-query averages:
  Avg nodes visited: 483.1
  Avg API calls: 77.4
  Avg edges examined: 4625.8

Timeout rate: 0.0%

Good: Low timeout rate. Safeguards are effective.


## 5. Show Best Path Examples

In [7]:
# Show best path examples
print("="*60)
print("BEST PATH EXAMPLES")
print("="*60)

if found_paths:
    # Sort by path length (prefer shorter paths)
    sorted_paths = sorted(found_paths, key=lambda r: r['path_length'])
    
    for result in sorted_paths[:5]:
        print(f"\n{'='*40}")
        print(f"From: {result['start_name']} ({result['start_id']})")
        print(f"To: {result['end_name']} ({result['end_id']})")
        print(f"Hops: {result['path_length']}")
        print(f"{'='*40}")
        
        for i, step in enumerate(result['path'], 1):
            print(f"  Step {i}: {step['subject']} --[{step['predicate']}]--> {step['object']}")
else:
    print("\nNo paths found between any entity pairs.")

BEST PATH EXAMPLES

From: glucose (CHEBI:4167)
To: water (CHEBI:15377)
Hops: 3
  Step 1: CHEBI:4167 --[biolink:subclass_of]--> D-glucose
  Step 2: CHEBI:17634 --[biolink:has_part]--> royal jelly
  Step 3: CHEBI:78665 --[biolink:has_part]--> water

From: NAD+ (CHEBI:15846)
To: water (CHEBI:15377)
Hops: 3
  Step 1: CHEBI:15846 --[biolink:related_to]--> NADH
  Step 2: CHEBI:16908 --[biolink:related_to]--> ALDH1A1
  Step 3: NCBIGene:216 --[biolink:interacts_with]--> water

From: NAD+ (CHEBI:15846)
To: L-leucine (CHEBI:16015)
Hops: 3
  Step 1: CHEBI:15846 --[biolink:related_to]--> NADH
  Step 2: CHEBI:16908 --[biolink:related_to]--> GLUD1
  Step 3: NCBIGene:2746 --[biolink:interacts_with]--> L-glutamic acid


## 6. Test with Different Parameters

In [8]:
# Compare semantic_only vs all edges
print("="*60)
print("SEMANTIC_ONLY COMPARISON")
print("="*60)

# Take a few pairs to compare
comparison_pairs = entity_pairs[:5]

for (start_id, start_name), (end_id, end_name) in comparison_pairs:
    print(f"\n{start_name} --> {end_name}:")
    
    # Semantic only
    result_semantic = find_path_bfs(
        start_id, end_id, max_hops=3, semantic_only=True
    )
    
    # All edges
    result_all = find_path_bfs(
        start_id, end_id, max_hops=3, semantic_only=False
    )
    
    print(f"  Semantic only: {result_semantic.termination_reason}, "
          f"path={len(result_semantic.path)}, "
          f"visited={result_semantic.stats.get('nodes_visited', 0)}")
    print(f"  All edges: {result_all.termination_reason}, "
          f"path={len(result_all.path)}, "
          f"visited={result_all.stats.get('nodes_visited', 0)}")

SEMANTIC_ONLY COMPARISON

glucose --> NAD+:


  Semantic only: exhausted, path=0, visited=419
  All edges: exhausted, path=0, visited=436

glucose --> cholesterol:


  Semantic only: exhausted, path=0, visited=419
  All edges: exhausted, path=0, visited=436

glucose --> alanine:


  Semantic only: exhausted, path=0, visited=419
  All edges: exhausted, path=0, visited=436

glucose --> ATP:


  Semantic only: exhausted, path=0, visited=419
  All edges: exhausted, path=0, visited=436

glucose --> water:
  Semantic only: found, path=3, visited=38
  All edges: found, path=3, visited=38


## 7. Save Multi-Hop Paths

In [9]:
# Save results
output_data = {
    'timestamp': datetime.now().isoformat(),
    'summary': {
        'total_pairs_tested': len(path_results),
        'paths_found': len(found_paths),
        'success_rate': len(found_paths) / len(path_results) if path_results else 0,
        'termination_reasons': dict(termination_reasons),
        'timeout_rate': termination_reasons.get('timeout', 0) / len(path_results) if path_results else 0,
        'total_nodes_visited': total_nodes_visited,
        'total_api_calls': total_api_calls,
    },
    'safeguards': {
        'max_visited_nodes': MAX_VISITED_NODES,
        'timeout_seconds': TIMEOUT_SECONDS,
        'max_queue_size': MAX_QUEUE_SIZE,
    },
    'paths': path_results,
}

save_json(output_data, OUTPUT_DIR / 'multi_hop_paths.json')
print(f"\nMulti-hop paths saved to: {OUTPUT_DIR / 'multi_hop_paths.json'}")


Multi-hop paths saved to: /home/trentleslie/Insync/projects/biomapper2/notebooks/kg_o1_v3/outputs/multi_hop_paths.json


## Summary

In [10]:
# Final summary
print("\n" + "="*60)
print("NOTEBOOK 04 COMPLETE")
print("="*60)

print(f"\nPath Finding Results:")
print(f"  - Pairs tested: {len(path_results)}")
print(f"  - Paths found: {len(found_paths)}")
print(f"  - Success rate: {100*len(found_paths)/len(path_results):.1f}%")
print(f"  - Timeout rate: {100*timeout_rate:.1f}%")

# Success criteria: find paths for 20+ pairs with <10% timeout rate
if len(found_paths) >= 5 and timeout_rate < 0.1:
    print(f"\nSuccess criteria met! Proceed to NB05.")
elif len(found_paths) > 0:
    print(f"\nPartial success. Path finding works but needs tuning.")
else:
    print(f"\nNo paths found. Graph may be too sparse for multi-hop reasoning.")

print(f"\nNext step: NB05 - Semantic QA Generation")


NOTEBOOK 04 COMPLETE

Path Finding Results:
  - Pairs tested: 20
  - Paths found: 3
  - Success rate: 15.0%
  - Timeout rate: 0.0%

Partial success. Path finding works but needs tuning.

Next step: NB05 - Semantic QA Generation
